# Realized Volatility Models in the OpenBB SDK

Within the Technical Analysis module, there are six models for calculating realized volatility.  Let's have a look at each one.

In [1]:
# Import statements

import pandas as pd
from openbb_terminal.sdk import openbb

## End Points

The cones function is for the rolling quantiles, while the other functions return historical calculations over a specific window. The `cones` function applies a `for` loop over many windows, grouping the results into selectable quantiles.

`windows = [3, 10, 30, 60, 90, 120, 150, 180, 210, 240, 300, 360]`

`openbb.ta.cones()`

`openbb.ta.rvol_garman_klass()`

`openbb.ta.rvol_hodges_tompkins()`

`openbb.ta.rvol_parkinson()`

`openbb.ta_rvol_rogers_satchell()`

`openbb.ta.rvol_std()`

`openbb.ta.rvol_yang_zhang()`

## Collect Some Data

One of the primary steps for most workflows is to capture historical price data. The most robust method is to use the `load` function from the `stocks` module. 

In [2]:
# This will obtain daily price data for the complete history of SPY. The start date only needs to be before the first trading day of SPY, no end date is required to obtain the full history.

df = openbb.stocks.load("SPY", start_date = '1990-01-01')

Loading Daily data for SPY with starting period 1993-01-29.

## Function Inputs and Parameters

The primary input for each function is a Pandas DataFrame containing OHLC data. All other parameters have default settings which do not require additional user input. The `is_crypto` boolean provides an easy switch for the number of trading-days-per-year. 252 is the standard for equities and trad-fi, while `is_crypto = True` enforces 365.


In [24]:
openbb.ta.standard_deviation?

Signature:     
openbb.ta.standard_deviation(
    data: pandas.core.frame.DataFrame,
    window: int = 30,
    trading_periods: Optional[int] = None,
    is_crypto: bool = False,
    clean: bool = True,
) -> pandas.core.frame.DataFrame
Call signature: openbb.ta.standard_deviation(*args: Any, **kwargs: Any) -> Any
Type:           standard_deviation
String form:    <openbb_terminal.common.technical_analysis.volatility_model.Operation object at 0x2ae778b80>
File:           ~/GitHub/OpenBBTerminal/openbb_terminal/common/technical_analysis/volatility_model.py
Docstring:     
Standard deviation measures how widely returns are dispersed from the average return.
It is the most common (and biased) estimator of volatility.

Parameters
----------
data : pd.DataFrame
    Dataframe of OHLC prices.
window : int [default: 30]
    Length of window to calculate over.
trading_periods : Optional[int] [default: 252]
    Number of trading periods in a year.
is_crypto : bool [default: False]
    If true, tr

## Sample Outputs

Let's take a look at what the expected outputs are. The data we have loaded already is the complete daily OHLC history of SPY. 

In [5]:
df.head(1)

,Open,High,Low,Close,Adj Close,Volume,Dividends,Stock Splits
date,,,,,,,,
1993-01-29,25.140198,25.140198,25.015122,25.12233,25.12233,1003200,0.0,0.0


### Standard Deviation Model

This model requires a minimum window of two trading periods, the default is thirty. The calculations, by default, are annualized over 252 trading-days-per-year. To change this behaviour:

- `is_crypto = True` to annualize over 365 days.
- Enter an integer value for the `trading_periods` argument. 

In [20]:
rvol = openbb.ta.rvol_std(df, window = 1).rename("Trading Period 252")
openbb.qa.line(
    data= rvol,
    title = "Rolling 30-Day Realized Vol - STD Model",
    log_y = False
)

Error: Window must be at least 2, defaulting to 30.


Let's compare the difference between annualizing over 252 trading days versus 365.

In [21]:
rvol = pd.concat([rvol, openbb.ta.rvol_std(df, window = 1, is_crypto = True, trading_periods = 365).rename("Trading Perdiod 365")], axis = 1)

openbb.forecast.plot(data = rvol, columns = rvol.columns)

Error: Window must be at least 2, defaulting to 30.
is_crypto is overridden by trading_periods.


### Comparing the Models Against Each Other

The other models work in exactly the same way, let's compare them against each other.  The default state for each model is a 30-day rolling window, annualized over 252 trading days.

In [23]:
rvol = pd.DataFrame()
rvol["STD Model"] = openbb.ta.standard_deviation(df)
rvol["Parkinson"] = openbb.ta.rvol_parkinson(df)
rvol["Hodges-Tompkins"] = openbb.ta.rvol_hodges_tompkins(df)
rvol["Garman-Klass"] = openbb.ta.rvol_garman_klass(df)
rvol["Rogers-Satchell"] = openbb.ta.rvol_rogers_satchell(df)
rvol["Yang-Zhang"] = openbb.ta.rvol_yang_zhang(df)

openbb.forecast.plot(data = rvol["2021-06-01":], columns = rvol.columns)

### Cones

This function generates the realized volatility quantiles. There are two functions, one is for returning the DataFrame only, the other is for returning a chart.

`openbb.ta.cones()`

`openbb.ta.cones_chart()`

For these functions, the lower and upper quantiles have been parameterized. Use the `lower_q` and `upper_q` arguments as a float between 0 and 1, representing the % value. The default model is standard deviation.

In [25]:
openbb.ta.cones(df)

,Realized,Min,Lower 25%,Median,Upper 75%,Max
3,0.067040,0.000014,0.064353,0.112163,0.186059,1.919684
10,0.090820,0.020060,0.088717,0.133404,0.193792,1.130634
30,0.133722,0.035120,0.101011,0.137059,0.199334,0.864874
60,0.120575,0.049786,0.107722,0.141835,0.200102,0.755843
90,0.114354,0.054997,0.110307,0.142355,0.202444,0.646355
120,0.108223,0.062658,0.110737,0.144314,0.206257,0.590355
150,0.100045,0.064545,0.112091,0.150400,0.206057,0.560031
180,0.096711,0.066184,0.111586,0.152975,0.209214,0.522603
210,0.094264,0.067330,0.112948,0.156266,0.212350,0.490692
240,0.091158,0.067542,0.113474,0.155415,0.218567,0.464773


In [26]:
openbb.ta.cones_chart(df, symbol = "SPY")

In [31]:
openbb.ta.cones_chart(df, model = 'Yang-Zhang', symbol = 'SPY')

The `data` input can also be the `load` function from the OpenBB SDK. Use the `symbol` argument to complete the title of the chart.

In [33]:
openbb.ta.cones_chart(
    data = openbb.stocks.load('SPY'),
    symbol = 'SPY',
    model = 'Garman-Klass',
    upper_q = 0.90,
    lower_q = 0.10
)

Loading Daily data for SPY with starting period 2020-05-19.

## Overlaying Against Implied Volatility.

SPY is a good symbol to work with because there is an index that tracks the 30-day ('ish) IV of the OTM options, VIX.  Using it as a proxy, we can compare historical implied versus realized volatility.  Similarly to how the DataFrame was constructed to compare all the models together, we can make a little function with a symbol input.

In [43]:
from typing import Optional

def rvol(data, window, trading_periods, is_crypto) -> pd.DataFrame:
    
    results = pd.DataFrame()

    results['Standard Deviation'] = openbb.ta.rvol_std(data, window, trading_periods, is_crypto)
    results['Parkinson'] = openbb.ta.rvol_parkinson(data, window, trading_periods, is_crypto)
    results['Hodges-Tompkins'] = openbb.ta.rvol_hodges_tompkins(data, window, trading_periods, is_crypto)
    results['Garman-Klass'] = openbb.ta.rvol_garman_klass(data, window, trading_periods, is_crypto)
    results['Rogers-Satchell'] = openbb.ta.rvol_rogers_satchell(data, window, trading_periods, is_crypto)
    results['Yang-Zhang'] = openbb.ta.rvol_yang_zhang(data, window, trading_periods, is_crypto)
    
    return results


def realized_vol(symbol, window:Optional[int] = 30, trading_periods:Optional[int] = None, is_crypto:Optional[bool] = False) -> pd.DataFrame:
    
    rvol_df = rvol(openbb.stocks.load(f"{symbol}"), window, trading_periods, is_crypto)
    
    return rvol_df


Now all models can be returned in a single DataFrame, with a variable input for the symbol.

In [46]:
data = realized_vol("SPY")

data

Loading Daily data for SPY with starting period 2020-05-19.

,Standard Deviation,Parkinson,Hodges-Tompkins,Garman-Klass,Rogers-Satchell,Yang-Zhang
date,,,,,,
2020-07-01,0.259206,0.187116,0.270157,0.182824,0.186237,0.274621
2020-07-02,0.255697,0.188072,0.266499,0.183940,0.187264,0.274203
2020-07-06,0.257207,0.186971,0.268073,0.182706,0.186137,0.276674
2020-07-07,0.259925,0.187828,0.270906,0.183770,0.187297,0.278017
2020-07-08,0.258707,0.184769,0.269637,0.180008,0.182682,0.267621
...,...,...,...,...,...,...
2023-05-18,0.127470,0.097234,0.132855,0.095223,0.096210,0.120858
2023-05-19,0.127335,0.096887,0.132715,0.095100,0.096174,0.120798
2023-05-22,0.127337,0.096409,0.132716,0.095345,0.096654,0.119689


Now let's add a column to the DataFrame for VIX and then plot them!

In [50]:
data["VIX"] = openbb.stocks.load("^VIX")["Adj Close"].rename("VIX")/100

Loading Daily data for ^VIX with starting period 2020-05-19.

In [52]:
fig = openbb.forecast.plot(data, columns = data.columns, external_axes = True)

In [61]:
fig.update_layout(
    {
    'title': 'SPY - 30-Day Realized vs. Implied Volatility',
    'title_y':0.95,
    'title_x':0.5,
    },
    legend=dict(
    yanchor="top",
    y=1,
    xanchor="right",
    orientation="h",
))

Experiment in combination with other functions, like the forecast models. 

In [87]:
%%capture
fig = openbb.forecast.nbeats_chart(
    data = data, 
    target_column = "VIX",
    past_covariates = "Standard Deviation,Parkinson,Hodges-Tompkins,Rogers-Satchell,Yang-Zhang",
    forecast_only = True,
    external_axes = True
)

In [88]:
fig.show()

The forecast above targets the realized volatility as past covariates to estimate the future values of VIX.  The forecast below is the same function and parameters, withouth the additional past covariates.

In [89]:
%%capture
fig = openbb.forecast.nbeats_chart(
    data = data, 
    target_column = "VIX",
    forecast_only = True,
    external_axes = True
)

In [90]:
fig.show()